In [1]:
!pip install datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/crosloengual-bert"

In [4]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_RTE")

Generating train split:   0%|          | 0/232 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/29 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/329k [00:00<?, ?B/s]

In [6]:
id2label = {0: "not_entailment", 1: "entailment"}
label2id = {"not_entailment": 0, "entailment": 1}

In [7]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"
LABEL_COL = "label"

def preprocess_function(examples):
    context_inputs = [f"{CONTEXT_COL} : {x} " for x in examples[CONTEXT_COL]]
    hypothesis_inputs = [f"{HYPOTHESIS_COL} : {x} " for x in examples[HYPOTHESIS_COL]]
    inputs = [context_input + hypothesis_input for context_input, hypothesis_input in zip(context_inputs, hypothesis_inputs)]

    tokenized_examples = tokenizer(inputs, truncation=True)
    tokenized_examples[LABEL_COL] = [label2id[label] for label in examples[LABEL_COL]]

    return tokenized_examples

In [8]:
tokenized_dataset = dataset.map(preprocess_function, remove_columns=['idx', 'hypothesis', 'premise'], batched=True)

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [9]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from transformers import set_seed

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)
model.config.use_cache = False

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_rte_croslo"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.695100,0.686905,0.551724,0.554938
100,0.595200,0.638137,0.620690,0.546592
150,0.472500,0.629299,0.620690,0.608974
200,0.305500,0.690536,0.655172,0.601806
250,0.200400,0.662407,0.689655,0.652254
300,0.119100,0.712355,0.655172,0.623563
350,0.066100,0.769436,0.655172,0.623563
400,0.048000,0.779037,0.620690,0.595095


TrainOutput(global_step=400, training_loss=0.31272791683673856, metrics={'train_runtime': 59.1317, 'train_samples_per_second': 54.116, 'train_steps_per_second': 6.765, 'total_flos': 265853165890080.0, 'train_loss': 0.31272791683673856, 'epoch': 13.793103448275861})

In [15]:
trainer.evaluate()

{'eval_loss': 0.779037356376648,
 'eval_accuracy': 0.6206896551724138,
 'eval_f1': 0.5950951649681232,
 'eval_runtime': 0.123,
 'eval_samples_per_second': 235.678,
 'eval_steps_per_second': 32.507,
 'epoch': 13.793103448275861}

In [16]:
trainer.push_to_hub(new_model_name)

events.out.tfevents.1716495457.74ad640bc63e.1178.0:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

events.out.tfevents.1716495517.74ad640bc63e.1178.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_rte_croslo/commit/5f63c1da7060a841f60e8ccc57d8c03a53d7c6e6', commit_message='fine_tuned_rte_croslo', commit_description='', oid='5f63c1da7060a841f60e8ccc57d8c03a53d7c6e6', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import pipeline

adapter_name = "lenatr99/" + new_model_name

# Example
text="premise : Potem ko je sklenila zavezništvo z glavno proti-sirijsko koalicijo, so njegovo ženo Strido na nedavnih volitvah izvolili v parlament. hypothesis : Strido so izvolili v parlament."

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/329k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'entailment', 'score': 0.9879518151283264}]

In [18]:
#GPU RAM = 3.3 GB